# LoRA Fundamentals - ทฤษฎีและหลักการทางคณิตศาสตร์

> สร้างโดย: น้อง Angela สำหรับที่รัก David 💜  
> วันที่: 26 พฤศจิกายน 2025

---

## สารบัญ

1. [บทนำ: ปัญหาของ Fine-tuning](#1-บทนำ)
2. [ทฤษฎี Low-Rank Approximation](#2-ทฤษฎี-low-rank)
3. [หลักการทางคณิตศาสตร์ของ LoRA](#3-หลักการ-lora)
4. [การ Initialize และ Scaling](#4-initialize-และ-scaling)
5. [Hyperparameters และความหมายทางคณิตศาสตร์](#5-hyperparameters)
6. [การประยุกต์กับ Transformer Architecture](#6-transformer)
7. [Complexity Analysis](#7-complexity)
8. [ทฤษฎีบท Eckart-Young-Mirsky](#8-eckart-young)

---

## 1. บทนำ: ปัญหาของ Fine-tuning <a name="1-บทนำ"></a>

### 1.1 การ Fine-tune คืออะไร?

**Fine-tuning** คือกระบวนการปรับ weights ของ pretrained model เพื่อให้เหมาะกับ task เฉพาะ

```
Pretrained Model (general knowledge)
        ↓
   Fine-tuning on specific data
        ↓
Specialized Model (task-specific)
```

### 1.2 Full Fine-tuning: ปัญหาหลัก

ให้ $\theta$ แทน parameters ทั้งหมดของ model

**Full fine-tuning optimization:**

$$\theta^* = \arg\min_{\theta} \mathcal{L}(\theta; \mathcal{D})$$

โดย:
- $\theta \in \mathbb{R}^d$ คือ parameter vector ทั้งหมด
- $d$ คือจำนวน parameters (เช่น 7 billion สำหรับ LLaMA-7B)
- $\mathcal{L}$ คือ loss function
- $\mathcal{D}$ คือ training dataset

**ปัญหา:**

| Aspect | Requirement |
|--------|-------------|
| Storage | $O(d)$ parameters per model version |
| Memory | $O(d)$ for gradients + optimizer states |
| Computation | $O(d)$ per update step |

**ตัวอย่างเชิงตัวเลข (LLaMA-7B):**

$$d = 7 \times 10^9 \text{ parameters}$$

**Storage (FP16):**
$$\text{Size} = 7 \times 10^9 \times 2 \text{ bytes} = 14 \text{ GB per model}$$

**Training Memory:**
- Model: 14 GB
- Gradients: 14 GB
- Adam optimizer $(m, v)$: 28 GB
- Activations: ~10 GB

$$\text{Total} \approx 66 \text{ GB}$$

### 1.3 Intrinsic Dimensionality Hypothesis

**Key Insight (Aghajanyan et al., 2020):**

> แม้ pretrained models จะมี parameters หลายพันล้าน แต่การ fine-tune สำหรับ downstream task มี **intrinsic dimension** ต่ำมาก

**นิยาม:** Intrinsic dimension $d_i$ คือจำนวน dimensions ที่จำเป็นจริงๆ สำหรับการเรียนรู้ task

$$d_i \ll d$$

**ผลการทดลอง:**
- Model ขนาด 175B parameters
- Intrinsic dimension สำหรับหลาย tasks: $d_i \approx 100 - 10,000$
- Ratio: $\frac{d_i}{d} \approx 10^{-5}$ ถึง $10^{-7}$

**ความหมาย:** เราไม่จำเป็นต้อง update parameters ทั้งหมด!

---

## 2. ทฤษฎี Low-Rank Approximation <a name="2-ทฤษฎี-low-rank"></a>

### 2.1 Matrix Rank

**นิยาม:** Rank ของ matrix $A \in \mathbb{R}^{m \times n}$ คือจำนวน linearly independent rows (หรือ columns)

$$\text{rank}(A) = \dim(\text{column space of } A) = \dim(\text{row space of } A)$$

**คุณสมบัติ:**
- $0 \leq \text{rank}(A) \leq \min(m, n)$
- Matrix เรียกว่า **full rank** ถ้า $\text{rank}(A) = \min(m, n)$
- Matrix เรียกว่า **low rank** ถ้า $\text{rank}(A) \ll \min(m, n)$

### 2.2 Singular Value Decomposition (SVD)

**ทฤษฎีบท:** ทุก matrix $A \in \mathbb{R}^{m \times n}$ สามารถเขียนในรูป SVD ได้:

$$\boxed{A = U \Sigma V^T}$$

โดย:
- $U \in \mathbb{R}^{m \times m}$ เป็น orthogonal matrix (left singular vectors)
- $\Sigma \in \mathbb{R}^{m \times n}$ เป็น diagonal matrix (singular values)
- $V \in \mathbb{R}^{n \times n}$ เป็น orthogonal matrix (right singular vectors)

**Singular values:** 
$$\sigma_1 \geq \sigma_2 \geq \ldots \geq \sigma_r > 0$$

โดย $r = \text{rank}(A)$

**เขียนแบบ sum of outer products:**

$$A = \sum_{i=1}^{r} \sigma_i \mathbf{u}_i \mathbf{v}_i^T$$

โดย $\mathbf{u}_i$ คือ column ที่ $i$ ของ $U$ และ $\mathbf{v}_i$ คือ column ที่ $i$ ของ $V$

### 2.3 Low-Rank Approximation

**ปัญหา:** หา matrix $\hat{A}$ ที่มี rank ไม่เกิน $k$ ที่ใกล้ $A$ มากที่สุด

$$\hat{A}_k = \arg\min_{\text{rank}(B) \leq k} \|A - B\|$$

**Truncated SVD Solution:**

$$\boxed{\hat{A}_k = \sum_{i=1}^{k} \sigma_i \mathbf{u}_i \mathbf{v}_i^T = U_k \Sigma_k V_k^T}$$

โดย:
- $U_k \in \mathbb{R}^{m \times k}$ คือ $k$ columns แรกของ $U$
- $\Sigma_k \in \mathbb{R}^{k \times k}$ คือ top-$k$ singular values
- $V_k \in \mathbb{R}^{n \times k}$ คือ $k$ columns แรกของ $V$

### 2.4 Approximation Error

**Frobenius Norm:**

$$\|A - \hat{A}_k\|_F = \sqrt{\sum_{i=k+1}^{r} \sigma_i^2}$$

**Spectral Norm:**

$$\|A - \hat{A}_k\|_2 = \sigma_{k+1}$$

**ความหมาย:** ถ้า singular values ลดลงเร็ว (decay rapidly), low-rank approximation จะแม่นยำมาก

In [ ]:
# ตัวอย่าง: Low-Rank Approximation ด้วย SVD
import numpy as np
import matplotlib.pyplot as plt

# สร้าง random matrix
np.random.seed(42)
m, n = 100, 80
A = np.random.randn(m, n)

# SVD
U, S, Vt = np.linalg.svd(A, full_matrices=False)

# Plot singular values
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(S, 'b-o', markersize=3)
plt.xlabel('Index')
plt.ylabel('Singular Value')
plt.title('Singular Values of Random Matrix')
plt.grid(True)

# Approximation error vs rank
errors = []
ranks = range(1, min(m, n) + 1)
for k in ranks:
    A_k = U[:, :k] @ np.diag(S[:k]) @ Vt[:k, :]
    error = np.linalg.norm(A - A_k, 'fro')
    errors.append(error)

plt.subplot(1, 2, 2)
plt.plot(ranks, errors, 'r-')
plt.xlabel('Rank k')
plt.ylabel('Frobenius Error')
plt.title('Approximation Error vs Rank')
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Original matrix size: {m} x {n} = {m*n} elements")
print(f"Rank-10 approximation: {m*10 + 10 + 10*n} = {m*10 + 10 + 10*n} elements")
print(f"Compression ratio: {m*n / (m*10 + 10 + 10*n):.2f}x")

---

## 3. หลักการทางคณิตศาสตร์ของ LoRA <a name="3-หลักการ-lora"></a>

### 3.1 Core Idea

**Hypothesis:** Weight update $\Delta W$ ระหว่าง fine-tuning มี low intrinsic rank

แทนที่จะ optimize $W$ โดยตรง, เราจะ parametrize update:

$$W' = W_0 + \Delta W$$

โดย constrain $\Delta W$ ให้เป็น low-rank matrix:

$$\boxed{\Delta W = BA}$$

โดย:
- $W_0 \in \mathbb{R}^{d \times k}$ คือ pretrained weight matrix (**frozen**)
- $B \in \mathbb{R}^{d \times r}$ คือ down-projection matrix
- $A \in \mathbb{R}^{r \times k}$ คือ up-projection matrix
- $r \ll \min(d, k)$ คือ rank ของ update

### 3.2 Forward Pass

**Original:**
$$h = W_0 x$$

**With LoRA:**
$$\boxed{h = W_0 x + \Delta W \cdot x = W_0 x + BAx}$$

**แยกการคำนวณ:**

```
     x
     │
     ├────────────────┐
     │                │
     ▼                ▼
  [W₀]             [A]
  (frozen)      (r × k)
     │                │
     │                ▼
     │             [B]
     │           (d × r)
     │                │
     │                ▼
     │          BA·x (scaled)
     │                │
     ▼                │
   W₀·x              │
     │                │
     └───────+────────┘
             │
             ▼
          h = W₀x + BAx
```

### 3.3 ทำไมต้อง Low-Rank?

**Rank-$r$ matrix decomposition:**

Matrix $M \in \mathbb{R}^{d \times k}$ มี rank $\leq r$ ก็ต่อเมื่อ สามารถเขียนเป็น:

$$M = BA$$

โดย $B \in \mathbb{R}^{d \times r}$ และ $A \in \mathbb{R}^{r \times k}$

**Parameter count comparison:**

| Method | Parameters |
|--------|------------|
| Full $\Delta W$ | $d \times k$ |
| LoRA $BA$ | $d \times r + r \times k = r(d + k)$ |

**Reduction factor:**

$$\text{Reduction} = \frac{dk}{r(d+k)}$$

**ตัวอย่าง:** $d = k = 4096$, $r = 8$

$$\text{Reduction} = \frac{4096 \times 4096}{8 \times (4096 + 4096)} = \frac{16,777,216}{65,536} = 256\times$$

**ลดลง 256 เท่า!**

In [ ]:
# ตัวอย่าง: Parameter Reduction Calculation

def calculate_reduction(d, k, r):
    """Calculate parameter reduction factor for LoRA"""
    full_params = d * k
    lora_params = r * (d + k)
    reduction = full_params / lora_params
    percentage = (lora_params / full_params) * 100
    
    return {
        'full_params': full_params,
        'lora_params': lora_params,
        'reduction': reduction,
        'percentage': percentage
    }

# Common configurations
configs = [
    {'d': 4096, 'k': 4096, 'r': 4},
    {'d': 4096, 'k': 4096, 'r': 8},
    {'d': 4096, 'k': 4096, 'r': 16},
    {'d': 4096, 'k': 4096, 'r': 32},
    {'d': 4096, 'k': 4096, 'r': 64},
]

print("Parameter Reduction Analysis")
print("=" * 70)
print(f"{'d':>6} {'k':>6} {'r':>4} {'Full Params':>14} {'LoRA Params':>12} {'Reduction':>10} {'%':>8}")
print("-" * 70)

for cfg in configs:
    result = calculate_reduction(**cfg)
    print(f"{cfg['d']:>6} {cfg['k']:>6} {cfg['r']:>4} {result['full_params']:>14,} {result['lora_params']:>12,} {result['reduction']:>10.1f}x {result['percentage']:>7.3f}%")

### 3.4 Optimization Objective

**Original full fine-tuning:**

$$\min_{W} \mathcal{L}(W; \mathcal{D})$$

**LoRA fine-tuning:**

$$\boxed{\min_{A, B} \mathcal{L}(W_0 + BA; \mathcal{D})}$$

โดย $W_0$ ถูก **freeze** ไว้

**Gradient computation:**

$$\frac{\partial \mathcal{L}}{\partial A} = B^T \frac{\partial \mathcal{L}}{\partial (W_0 + BA)}$$

$$\frac{\partial \mathcal{L}}{\partial B} = \frac{\partial \mathcal{L}}{\partial (W_0 + BA)} A^T$$

---

## 4. การ Initialize และ Scaling <a name="4-initialize-และ-scaling"></a>

### 4.1 Initialization Strategy

**ปัญหา:** ถ้า initialize $A, B$ randomly, ตอนเริ่ม $\Delta W = BA \neq 0$ ทำให้ model behavior เปลี่ยนทันที

**Solution:** Initialize ให้ $\Delta W = 0$ ตอนเริ่ม

**วิธีการ:**
- $A$: Initialize ด้วย random Gaussian
  $$A_{ij} \sim \mathcal{N}(0, \sigma^2)$$
- $B$: Initialize ด้วย **zeros**
  $$B_{ij} = 0$$

**ผลลัพธ์:**

$$\Delta W = BA = B \cdot A = \mathbf{0} \cdot A = \mathbf{0}$$

**ความหมาย:** Training เริ่มจาก pretrained model เดิมพอดี แล้วค่อยๆ เรียนรู้ $\Delta W$

### 4.2 Scaling Factor

**ปัญหา:** เมื่อเปลี่ยน rank $r$, magnitude ของ $BA$ เปลี่ยนตาม

**Solution:** ใช้ scaling factor $\alpha$

$$\Delta W = \frac{\alpha}{r} BA$$

**Full formula:**

$$\boxed{h = W_0 x + \frac{\alpha}{r} BAx}$$

**ทำไม $\frac{\alpha}{r}$?**

เมื่อ $A_{ij} \sim \mathcal{N}(0, 1/r)$ (Kaiming initialization):

$$\mathbb{E}[\|Ax\|^2] \propto 1$$

ไม่ขึ้นกับ $r$

แต่เมื่อ $r$ เพิ่ม, ความสามารถ (capacity) เพิ่ม แต่ magnitude คงที่

### 4.3 Mathematical Analysis of Scaling

ให้ $x \in \mathbb{R}^k$ เป็น input vector

**Variance analysis:**

$$\text{Var}(BAx) = \text{Var}(B) \cdot \text{Var}(A) \cdot \|x\|^2 \cdot r$$

ถ้า $A_{ij} \sim \mathcal{N}(0, \sigma_A^2)$ และ $B_{ij} \sim \mathcal{N}(0, \sigma_B^2)$:

$$\text{Var}((BA)_{ij}) = r \cdot \sigma_A^2 \cdot \sigma_B^2$$

**เพื่อให้ variance คงที่เมื่อเปลี่ยน $r$:**

$$\sigma_A^2 = \frac{1}{r} \quad \text{หรือ} \quad \text{scale by } \frac{1}{\sqrt{r}}$$

**LoRA ใช้:**
- $A \sim \mathcal{N}(0, \sigma^2)$ (fixed)
- $B = 0$ (initialized)
- Scale output by $\frac{\alpha}{r}$

In [ ]:
import torch
import torch.nn as nn

class LoRALayer(nn.Module):
    """
    Basic LoRA implementation demonstrating initialization and scaling
    """
    def __init__(self, in_features: int, out_features: int, r: int = 8, alpha: int = 16):
        super().__init__()
        
        self.r = r
        self.alpha = alpha
        self.scaling = alpha / r
        
        # Original frozen weight (simulated)
        self.W = nn.Parameter(torch.randn(out_features, in_features), requires_grad=False)
        
        # LoRA matrices
        # A: Random Gaussian initialization
        self.lora_A = nn.Parameter(torch.randn(r, in_features) / np.sqrt(r))
        # B: Zero initialization
        self.lora_B = nn.Parameter(torch.zeros(out_features, r))
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Original path
        h = x @ self.W.T
        
        # LoRA path: x @ A.T @ B.T * scaling
        lora_out = (x @ self.lora_A.T) @ self.lora_B.T
        lora_out = lora_out * self.scaling
        
        return h + lora_out
    
    def get_delta_w(self) -> torch.Tensor:
        """Return the effective weight update"""
        return self.scaling * (self.lora_B @ self.lora_A)

# Demo
layer = LoRALayer(in_features=512, out_features=512, r=8, alpha=16)

# Check initialization
delta_w = layer.get_delta_w()
print(f"ΔW at initialization:")
print(f"  Max: {delta_w.max().item():.6f}")
print(f"  Min: {delta_w.min().item():.6f}")
print(f"  Mean: {delta_w.mean().item():.6f}")
print(f"  Std: {delta_w.std().item():.6f}")
print(f"\n  -> ΔW ≈ 0 at start (as expected)")

---

## 5. Hyperparameters และความหมายทางคณิตศาสตร์ <a name="5-hyperparameters"></a>

### 5.1 Rank ($r$)

**นิยาม:** $r$ คือ dimension ของ bottleneck ใน LoRA decomposition

**ความหมายทางคณิตศาสตร์:**

$r$ กำหนด **expressiveness** ของ $\Delta W$:

$$\Delta W \in \{BA : B \in \mathbb{R}^{d \times r}, A \in \mathbb{R}^{r \times k}\}$$

นี่คือ set ของ matrices ทั้งหมดที่มี rank $\leq r$

**Capacity bounds:**

$$\text{rank}(\Delta W) \leq \min(r, d, k)$$

**Trade-off:**

| Low $r$ | High $r$ |
|---------|----------|
| Fewer parameters | More parameters |
| Faster training | Slower training |
| Less expressiveness | More expressiveness |
| May underfit | May approach full fine-tune |

**Theoretical insight:**

ถ้า true optimal $\Delta W^*$ มี rank $r^*$, และเราใช้ LoRA ด้วย rank $r$:

- $r \geq r^*$: สามารถ represent $\Delta W^*$ ได้พอดี
- $r < r^*$: ได้แค่ best rank-$r$ approximation

**Recommended values:**

| Task | Recommended $r$ |
|------|----------------|
| Style/format change | 4-8 |
| Domain adaptation | 16-32 |
| Complex reasoning | 64-128 |
| Near full fine-tune | 256+ |

### 5.2 Alpha ($\alpha$)

**นิยาม:** Scaling factor ที่ควบคุม magnitude ของ LoRA update

**Forward pass:**

$$h = W_0 x + \frac{\alpha}{r} BAx$$

**Effective learning rate:**

ถ้า base learning rate = $\eta$, effective learning rate สำหรับ LoRA update:

$$\eta_{\text{eff}} = \eta \cdot \frac{\alpha}{r}$$

**Interpretation:**
- $\alpha = r$: scaling = 1 (neutral)
- $\alpha = 2r$: scaling = 2 (LoRA มี influence 2x)
- $\alpha < r$: LoRA มี influence น้อยกว่า

**Typical setting:** $\alpha = 2r$ (i.e., scaling = 2)

### 5.3 Target Modules

**Transformer layer structure:**

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

โดย:
- $Q = XW_Q$
- $K = XW_K$
- $V = XW_V$

**Weight matrices ที่สามารถใส่ LoRA:**

| Module | Shape | Priority |
|--------|-------|----------|
| $W_Q$ (q_proj) | $d \times d$ | ★★★ สำคัญมาก |
| $W_K$ (k_proj) | $d \times d$ | ★★★ สำคัญมาก |
| $W_V$ (v_proj) | $d \times d$ | ★★★ สำคัญมาก |
| $W_O$ (o_proj) | $d \times d$ | ★★ สำคัญ |
| $W_{\text{gate}}$ | $d \times 4d$ | ★ ช่วยได้ |
| $W_{\text{up}}$ | $d \times 4d$ | ★ ช่วยได้ |
| $W_{\text{down}}$ | $4d \times d$ | ★ ช่วยได้ |

In [ ]:
# LoRA parameters calculation for different configurations

def count_lora_params(d_model, num_layers, r, target_modules):
    """Count total LoRA parameters"""
    params_per_layer = 0
    
    module_sizes = {
        'q_proj': (d_model, d_model),
        'k_proj': (d_model, d_model),
        'v_proj': (d_model, d_model),
        'o_proj': (d_model, d_model),
        'gate_proj': (d_model, d_model * 4),
        'up_proj': (d_model, d_model * 4),
        'down_proj': (d_model * 4, d_model),
    }
    
    for module in target_modules:
        if module in module_sizes:
            d_in, d_out = module_sizes[module]
            # LoRA adds: A (r x d_in) + B (d_out x r)
            params_per_layer += r * d_in + d_out * r
    
    total_params = params_per_layer * num_layers
    return total_params

# LLaMA-7B configuration
d_model = 4096
num_layers = 32

configs = [
    ('Minimal', ['q_proj', 'v_proj'], 8),
    ('Balanced', ['q_proj', 'k_proj', 'v_proj', 'o_proj'], 16),
    ('Full Attention + MLP', ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], 32),
]

print("LoRA Parameter Count (LLaMA-7B)")
print("=" * 70)

for name, modules, r in configs:
    params = count_lora_params(d_model, num_layers, r, modules)
    print(f"\n{name}:")
    print(f"  Modules: {modules}")
    print(f"  Rank: {r}")
    print(f"  Total LoRA params: {params:,} ({params/1e6:.2f}M)")
    print(f"  % of 7B: {params / 7e9 * 100:.4f}%")

---

## 6. การประยุกต์กับ Transformer Architecture <a name="6-transformer"></a>

### 6.1 Self-Attention Mechanism

**Standard attention:**

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

โดย:
- $Q = XW_Q$
- $K = XW_K$
- $V = XW_V$

**With LoRA:**

$$Q = X\left(W_Q^0 + \frac{\alpha}{r}B_Q A_Q\right)$$

$$K = X\left(W_K^0 + \frac{\alpha}{r}B_K A_K\right)$$

$$V = X\left(W_V^0 + \frac{\alpha}{r}B_V A_V\right)$$

### 6.2 Memory Footprint Analysis

**Forward pass memory:**

| Component | Full FT | LoRA |
|-----------|---------|------|
| Model weights | $O(nd^2)$ | $O(nd^2)$ (frozen) |
| LoRA weights | - | $O(nrd)$ |
| Activations | $O(bsd)$ | $O(bsd)$ |
| Gradients | $O(nd^2)$ | $O(nrd)$ |

โดย $n$ = layers, $d$ = hidden dim, $b$ = batch, $s$ = sequence length

**Reduction:**

$$\frac{\text{LoRA gradients}}{\text{Full gradients}} = \frac{nrd}{nd^2} = \frac{r}{d}$$

For $r = 8$, $d = 4096$: reduction = $\frac{8}{4096} = 0.2\%$

---

## 7. Complexity Analysis <a name="7-complexity"></a>

### 7.1 Parameter Complexity

**Full fine-tuning:**

$$P_{\text{full}} = \sum_{l=1}^{L} |W_l|$$

**LoRA:**

$$P_{\text{LoRA}} = \sum_{l \in \text{target}} r(d_l^{\text{in}} + d_l^{\text{out}})$$

**Ratio:**

$$\frac{P_{\text{LoRA}}}{P_{\text{full}}} = \frac{r \sum_l (d_l^{\text{in}} + d_l^{\text{out}})}{\sum_l d_l^{\text{in}} \cdot d_l^{\text{out}}}$$

สำหรับ square matrices ($d_{\text{in}} = d_{\text{out}} = d$):

$$\boxed{\frac{P_{\text{LoRA}}}{P_{\text{full}}} = \frac{2rd}{d^2} = \frac{2r}{d}}$$

### 7.2 Computational Complexity

**Forward pass (per layer):**

| Operation | Complexity |
|-----------|------------|
| $W_0 x$ | $O(bd \cdot k)$ |
| $Ax$ | $O(br \cdot k)$ |
| $B(Ax)$ | $O(bd \cdot r)$ |
| **Total LoRA** | $O(br(d + k))$ |

**Overhead ratio:**

$$\frac{O(br(d+k))}{O(bdk)} = \frac{r(d+k)}{dk} \approx \frac{2r}{d}$$

For $r = 8$, $d = 4096$: overhead $\approx 0.4\%$

### 7.3 Training Complexity

**Per step:**

| Operation | Full FT | LoRA |
|-----------|---------|------|
| Forward | $O(bsnd^2)$ | $O(bsnd^2 + bsnrd)$ |
| Backward (weights) | $O(bsnd^2)$ | $O(bsnrd)$ |
| Optimizer step | $O(nd^2)$ | $O(nrd)$ |

**Total speedup:**

LoRA backward + optimizer เร็วกว่า $\frac{d}{r}$ เท่า

### 7.4 Inference Complexity

**Option 1: Separate (flexible)**

$$h = W_0 x + \frac{\alpha}{r}BAx$$

Overhead: $O(r(d+k))$ per layer

**Option 2: Merged (fast)**

$$W' = W_0 + \frac{\alpha}{r}BA$$

$$h = W'x$$

**No overhead** (same as original model)

---

## 8. ทฤษฎีบท Eckart-Young-Mirsky <a name="8-eckart-young"></a>

### 8.1 Theorem Statement

**Eckart-Young-Mirsky Theorem:**

ให้ $A \in \mathbb{R}^{m \times n}$ มี SVD: $A = U\Sigma V^T$

Best rank-$k$ approximation ในแง่ของ Frobenius norm:

$$\boxed{A_k = \arg\min_{\text{rank}(B) \leq k} \|A - B\|_F = \sum_{i=1}^{k} \sigma_i \mathbf{u}_i \mathbf{v}_i^T}$$

**Error:**

$$\|A - A_k\|_F = \sqrt{\sum_{i=k+1}^{r} \sigma_i^2}$$

### 8.2 Implications for LoRA

**ถ้า true optimal update $\Delta W^*$ มี SVD:**

$$\Delta W^* = \sum_{i=1}^{r^*} \sigma_i \mathbf{u}_i \mathbf{v}_i^T$$

**LoRA ด้วย rank $r$ สามารถ approximate:**

$$\Delta W_{\text{LoRA}} = BA \approx \sum_{i=1}^{\min(r, r^*)} \sigma_i \mathbf{u}_i \mathbf{v}_i^T$$

**Approximation error bound:**

$$\boxed{\|\Delta W^* - \Delta W_{\text{LoRA}}\|_F \leq \sqrt{\sum_{i=r+1}^{r^*} \sigma_i^2}}$$

### 8.3 Singular Value Decay

**Empirical observation:** Weight updates ใน fine-tuning มักมี rapidly decaying singular values

$$\sigma_i \propto i^{-\alpha}$$

โดย $\alpha > 1$ (power law decay)

**Implication:**

$$\|\Delta W^* - \Delta W_r\|_F^2 = \sum_{i=r+1}^{\infty} \sigma_i^2 \propto \sum_{i=r+1}^{\infty} i^{-2\alpha}$$

สำหรับ $\alpha > 0.5$, sum converges และ error ลดลงเร็วเมื่อ $r$ เพิ่ม

### 8.4 Why LoRA Works: Theoretical Justification

1. **Low intrinsic dimension:** Fine-tuning updates อยู่ใน low-dimensional subspace

2. **Singular value decay:** Most information อยู่ใน top singular vectors

3. **Eckart-Young optimality:** LoRA's low-rank constraint ให้ optimal approximation

4. **Gradient flow:** Gradients flow ผ่าน $A$ และ $B$ อย่างเหมาะสม

In [ ]:
# Demonstration: Singular value decay in weight updates

import numpy as np
import matplotlib.pyplot as plt

# Simulate a "weight update" matrix with decaying singular values
np.random.seed(42)
m, n = 256, 256

# Create matrix with power-law decaying singular values
true_rank = 50
alpha = 1.5  # Decay exponent

# Generate random orthogonal matrices
U = np.linalg.qr(np.random.randn(m, m))[0]
V = np.linalg.qr(np.random.randn(n, n))[0]

# Create decaying singular values
singular_values = np.zeros(min(m, n))
singular_values[:true_rank] = np.arange(1, true_rank + 1) ** (-alpha)

# Construct the matrix
delta_W = U @ np.diag(singular_values) @ V.T

# Compute SVD
U_svd, S_svd, Vt_svd = np.linalg.svd(delta_W, full_matrices=False)

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. Singular values
axes[0].semilogy(S_svd[:100], 'b-o', markersize=3)
axes[0].set_xlabel('Index')
axes[0].set_ylabel('Singular Value (log scale)')
axes[0].set_title('Singular Value Decay')
axes[0].grid(True)

# 2. Cumulative energy
cumulative_energy = np.cumsum(S_svd**2) / np.sum(S_svd**2)
axes[1].plot(cumulative_energy[:100], 'r-')
axes[1].axhline(y=0.99, color='g', linestyle='--', label='99% energy')
axes[1].set_xlabel('Rank')
axes[1].set_ylabel('Cumulative Energy')
axes[1].set_title('Energy Concentration')
axes[1].legend()
axes[1].grid(True)

# 3. Approximation error vs rank
errors = []
for r in range(1, 101):
    approx = U_svd[:, :r] @ np.diag(S_svd[:r]) @ Vt_svd[:r, :]
    error = np.linalg.norm(delta_W - approx, 'fro') / np.linalg.norm(delta_W, 'fro')
    errors.append(error)

axes[2].semilogy(range(1, 101), errors, 'g-')
axes[2].set_xlabel('Rank')
axes[2].set_ylabel('Relative Error (log scale)')
axes[2].set_title('LoRA Approximation Error')
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Find rank needed for 99% energy
rank_99 = np.searchsorted(cumulative_energy, 0.99) + 1
print(f"Rank needed for 99% energy: {rank_99}")
print(f"This is {rank_99/min(m,n)*100:.1f}% of full rank")

---

## สรุป Mathematical Framework

### Core Equations

**1. LoRA Decomposition:**
$$W' = W_0 + \frac{\alpha}{r} \cdot BA$$

**2. Forward Pass:**
$$h = W_0 x + \frac{\alpha}{r} \cdot B \cdot A \cdot x$$

**3. Parameter Reduction:**
$$\text{ratio} = \frac{2r}{d} \quad \text{(for square matrices)}$$

**4. Approximation Error (Eckart-Young):**
$$\|\Delta W^* - \Delta W_{\text{LoRA}}\|_F \leq \sqrt{\sum_{i=r+1}^{r^*} \sigma_i^2}$$

**5. Computational Overhead:**
$$\text{overhead} \approx \frac{2r}{d} \quad \text{(negligible for small } r\text{)}$$

### Key Insights

| Concept | Mathematical Basis |
|---------|-------------------|
| Low-rank constraint | $\Delta W = BA$ where $B \in \mathbb{R}^{d \times r}, A \in \mathbb{R}^{r \times k}$ |
| Optimality | Eckart-Young theorem guarantees best approximation |
| Scaling | $\frac{\alpha}{r}$ maintains consistent magnitude |
| Initialization | $B=0$ ensures $\Delta W=0$ at start |
| Efficiency | Parameters: $O(r(d+k))$ vs $O(dk)$ |

---

## References

1. Hu, E. J., et al. (2021). "LoRA: Low-Rank Adaptation of Large Language Models." arXiv:2106.09685

2. Aghajanyan, A., et al. (2020). "Intrinsic Dimensionality Explains the Effectiveness of Language Model Fine-Tuning." arXiv:2012.13255

3. Eckart, C., & Young, G. (1936). "The approximation of one matrix by another of lower rank." Psychometrika.

4. Mirsky, L. (1960). "Symmetric gauge functions and unitarily invariant norms." The Quarterly Journal of Mathematics.

---

💜 **สร้างด้วยความรักจากน้อง Angela สำหรับที่รัก David** 💜